In [1]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 81991, done.
remote: Counting objects: 100% (458/458), done.
remote: Compressing objects: 100% (203/203), done.
remote: Total 81991 (delta 293), reused 400 (delta 255), pack-reused 81533
Receiving objects: 100% (81991/81991), 596.36 MiB | 20.20 MiB/s, done.
Resolving deltas: 100% (58492/58492), done.


In [2]:
%%bash
cd models/research
# Compile protos.
protoc object_detection/protos/*.proto --python_out=.
# Install TensorFlow Object Detection API.
cp object_detection/packages/tf2/setup.py .
python -m pip install --use-feature=2020-resolver .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/models/research
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 70.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.1/352.1 KB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 KB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.6/116.6 KB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.9/238.9 KB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [3]:
import matplotlib
import matplotlib.pyplot as plt
import os
import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage
import tensorflow as tf
from google.colab import files
import pathlib
import itertools
import random
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import config_util
from object_detection.utils import colab_utils
from object_detection.builders import model_builder
%matplotlib inline

In [4]:
!git clone https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md

Cloning into 'tf2_detection_zoo.md'...
fatal: repository 'https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md/' not found


In [5]:
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=1367adab02c6757b3180c608ea1112ec289b5997e956f2c29743a74069c9c016
  Stored in directory: /root/.cache/pip/wheels/04/5f/3e/46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget


In [6]:
import wget
model_link = "http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz"
wget.download(model_link)

'faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz'

In [7]:
import tarfile
tar = tarfile.open('/content/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz')
tar.extractall('.') 
tar.close()

In [8]:
#Baje el ZIP de GUNS y descomprimalo en el content/
%cd /content/
!unzip gunniest.zip  
!unzip /content/gunniest/gunniest.zip  -d /content/gunniest

/content
Archive:  gunniest.zip
  inflating: gunniest/armas (1).jpg  
  inflating: gunniest/armas (10).jpg  
  inflating: gunniest/armas (100).jpg  
  inflating: gunniest/armas (101).jpg  
  inflating: gunniest/armas (102).jpg  
  inflating: gunniest/armas (103).jpg  
  inflating: gunniest/armas (104).jpg  
  inflating: gunniest/armas (105).jpg  
  inflating: gunniest/armas (106).jpg  
  inflating: gunniest/armas (107).jpg  
  inflating: gunniest/armas (108).jpg  
  inflating: gunniest/armas (109).jpg  
  inflating: gunniest/armas (11).jpg  
  inflating: gunniest/armas (110).jpg  
  inflating: gunniest/armas (111).jpg  
  inflating: gunniest/armas (112).jpg  
  inflating: gunniest/armas (113).jpg  
  inflating: gunniest/armas (114).jpg  
  inflating: gunniest/armas (115).jpg  
  inflating: gunniest/armas (116).jpg  
  inflating: gunniest/armas (117).jpg  
  inflating: gunniest/armas (118).jpg  
  inflating: gunniest/armas (119).jpg  
  inflating: gunniest/armas (12).jpg  
  inflating: 

In [9]:
!pip install annotated_images

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for annotated_images: filename=annotated_images-0.1.4-py3-none-any.whl size=4290 sha256=9c9fd2617ec96a4884bf164a8a832136f45901a1187d009097b4a17c43a7ed89
  Stored in directory: /root/.cache/pip/wheels/11/e5/f4/a3bfa91aee3dc301acceaca2289bcc86f768358ac0d6dce39f
Successfully built annotated_images


In [10]:
import annotated_images

In [13]:
annotated_images.split('/content/gunniest','/content/gunniest_splt/')

TypeError: ignored